In [1]:
include("../src/CMPBP.jl")
using .CMPBP
using LinearAlgebra

const CMPS{F,N} = Array{F,N} where {F,N};

In [ ]:
function onebpiter_infinite_regular!(A::CMPS{F,5}, factor, prob;
    maxiter=100, tol=1e-12, maxit_ascent=100, tol_ascent=1e-12, η=0.1 , maxit_pow=1000, tol_pow=1e-12) where F<:Number
    q,d = size(A,1), size(A,4)
    size(A,2)==size(A,3)==q && size(A,5)==d || error("Wrong dimensions for A, got $(size(A))")

    E(x,y; q=2) = [i==x && j==y for i in 1:q, j in 1:q]

    Anew = rand(size(A)...)
    AA = zeros(F, q,q,q,(q*d)^2,(q*d)^2)
    for it in 1:maxiter
        for x in 1:q, y in 1:q, z in 1:q
            AA[x,y,z,:,:] = sum(prob(factor, y,x,x₁ᵗ,x₂ᵗ,z)*kron(E(x₁ᵗ,x₁ᵗ), E(x₂ᵗ,x₂ᵗ), I(d), I(d)) for x₁ᵗ in 1:q, x₂ᵗ in 1:q)
            if x==y
                AA[x,y,z,:,:] .+= (sum(kron(I(q), E(x₂ᵗ,x₂ᵗ⁺¹;q), I(d), A[x₂ᵗ,x₂ᵗ⁺¹,x,:,:]) for x₂ᵗ in 1:q, x₂ᵗ⁺¹ in 1:q) +
                                sum(kron(E(x₁ᵗ,x₁ᵗ⁺¹; q), I(q), A[x₁ᵗ,x₁ᵗ⁺¹,x,:,:], I(d)) for x₁ᵗ in 1:q, x₁ᵗ⁺¹ in 1:q))
            end
        end
        ascent!(Anew,AA; maxiter=maxit_ascent, η=η, tol=tol_ascent, maxiter_pow=maxit_pow, tol_pow=tol_pow)
        @show marginals(A) marginals(Anew)
        if norm(Anew - A) < tol
            return it, Anew
        end
        A .= Anew
    end
    return maxiter, Anew
end

onebpiter_infinite_regular! (generic function with 1 method)

In [5]:
q = 2
d = 5

A = rand(q,q,q,d,d)

k = 3
β = 1.0
J = 0.2
h = 0.1


function prob_glauber(factor, y,x,x∂i...; q=2)
    β, J, h = factor.β, factor.J, factor.h
    localfield = β*(h + J*sum(potts2spin, x∂i))
    if x!=y
        return exp(potts2spin(y)*localfield) / (2*cosh(localfield))
    else
        return -sum(exp(potts2spin(y)*localfield) / (2*cosh(localfield)) for y in 1:q if y≠x)
    end
end

struct GlauberFactor{F}
    β::F
    J::F
    h::F
end

w = GlauberFactor(β, J, h)
;

In [9]:
onebpiter_infinite_regular!(A, w, prob_glauber; maxiter=30, tol=1e-12, maxit_ascent=10^4, tol_ascent=1e-12, η=0.1, maxit_pow=10^4, tol_pow=1e-12);

marginals(A) = ComplexF64[0.41659749664367157 + 1.228885888161515e-16im, 0.5834025033563284 - 1.228885888161515e-16im]
marginals(Anew) = ComplexF64[0.40679219114814336 - 1.5620311760321713e-17im, 0.5932078088518566 + 1.562031176032171e-17im]
marginals(A) = ComplexF64[0.40679219114814336 - 1.5620311760321713e-17im, 0.5932078088518566 + 1.562031176032171e-17im]
marginals(Anew) = ComplexF64[0.42126078054410526 - 5.036757987981735e-17im, 0.5787392194558947 + 5.036757987981735e-17im]
marginals(A) = ComplexF64[0.42126078054410526 - 5.036757987981735e-17im, 0.5787392194558947 + 5.036757987981735e-17im]
marginals(Anew) = ComplexF64[0.43580535674880627 - 5.612719063007141e-17im, 0.5641946432511938 + 5.612719063007144e-17im]
marginals(A) = ComplexF64[0.43580535674880627 - 5.612719063007141e-17im, 0.5641946432511938 + 5.612719063007144e-17im]
marginals(Anew) = ComplexF64[0.4225074555221004 - 1.5868010410226068e-17im, 0.5774925444778997 + 1.5868010410226068e-17im]
marginals(A) = ComplexF64[0.42250

InterruptException: InterruptException:

In [10]:
magnetization(A)

0.20662335992905845 + 1.325727722786343e-16im

In [11]:
using MatrixProductBP.Models
g = Models.RandomRegular(3)
equilibrium_observables(g, J; β, h)

(m = 0.1939666644448763, r = 0.22272681103886016, e = -0.08621470975614569)